<a href="https://colab.research.google.com/github/federicOO1/LAB-IA/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## import librerie


In [1]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets
import matplotlib.pyplot as plt
from torchvision import transforms
import os
import numpy as np
import PIL
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')

In [2]:
!pip install rasterio
import rasterio
from rasterio.plot import reshape_as_image

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
os.chdir("/content/drive/MyDrive/PotsdamDataset")

In [5]:
class PotsdamDataset(Dataset):
    def __init__(self, dataset_folder):
        self.dataset_folder = dataset_folder
        self.image_paths = []
        self.world_file_paths = []
        self.mask_paths = []

        # Leggi i percorsi delle immagini, dei file .tfw e delle maschere
        end_RGBIR_folder = os.listdir(dataset_folder)[0]
        end_LABELS_folder = os.listdir(dataset_folder)[1]

        RGBIR_folder = data_folder + '/' + end_RGBIR_folder
        LABELS_folder = data_folder + '/' + end_LABELS_folder

        for file_name in os.listdir(RGBIR_folder):
            if file_name.endswith('.tif'):
                image_path = os.path.join(RGBIR_folder, file_name)
                world_file_path = os.path.join(RGBIR_folder, file_name.replace('.tif', '.tfw'))

                if os.path.exists(world_file_path):
                    self.image_paths.append(image_path)
                    self.world_file_paths.append(world_file_path)


        for label_name in os.listdir(LABELS_folder):
            mask_path = os.path.join(LABELS_folder, label_name)

            if os.path.exists(mask_path):
              self.mask_paths.append(mask_path)
        self.image_paths.sort()
        self.world_file_paths.sort()
        self.mask_paths.sort()

    def __len__(self):
        return len(self.image_paths)

    def get_image_paths(self, indices):
        return [self.image_paths[idx] for idx in indices]

    def get_mask_paths(self, indices):
        return [self.mask_paths[idx] for idx in indices]

    def load_world_file(self, world_file_path):
          lines = open(world_file_path).readlines()
          try:
              parameters = [float(line.strip()) for line in lines if line.strip()]
              if len(parameters) == 6:
                  return parameters
              else:
                  raise ValueError("Il file .tfw non contiene 6 parametri.")
          except Exception as e:
              print(f"Errore durante la lettura dei parametri di georeferenziazione: {str(e)}")
              return None

    def convert_labels_to_tensor(self, rgb_label):
      #print("Current rgb_label:",rgb_label)
      # Definisci i colori delle classi nella maschera RGB
      colors_to_labels = {
          (255, 255, 255): 0,  # Impervious surfaces -> Classe 0
          (0, 0, 255): 1,  # Building -> Classe 1
          (0, 255, 255): 2,  # Low vegetation -> Classe 2
          (0, 255, 0): 3,  # Tree -> Classe 3
          (255, 255, 0): 4,  # Car -> Classe 4
          (255, 0, 0): 5  # Clutter/background -> Classe 5
      }
      color_image = np.array(rgb_label)

      # Rappresenta color_array come un array (6, 3)
      color_array = np.array(list(colors_to_labels.keys()),dtype=np.uint8)

      # Reshape l'immagine per renderla compatibile con l'operazione di broadcasting
      color_image_reshaped = color_image.reshape(-1, 3)

      # Calcola le differenze tra i colori nella maschera RGB e i colori delle classi
      color_diffs = np.sum(np.abs(color_image_reshaped[:, None, :] - color_array[None, :, :]), axis=-1)

      # Trova l'indice del colore più vicino per ciascun pixel
      closest_color_indices = np.argmin(color_diffs, axis=-1)

      # Mappa gli indici di colore ai valori delle classi
      class_label = np.where(np.min(color_diffs, axis=-1) == 0, closest_color_indices, -1)

      # Reshape per tornare alle dimensioni dell'immagine originale
      class_label = class_label.reshape(rgb_label.shape[1], rgb_label.shape[2])

      # Crea il tensore delle etichette di classe
      class_label_tensor = torch.tensor(class_label, dtype=torch.long)

      return class_label_tensor

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        world_file_path = self.world_file_paths[idx]
        mask_path = self.mask_paths[idx]

        # Carica l'immagine TIFF utilizzando la libreria rasterio
        image = rasterio.open(image_path).read()

        # Carica i parametri di georeferenziazione dal file .tfw
        world_params = self.load_world_file(world_file_path)

        # Carica la maschera con rasterio
        mask = rasterio.open(mask_path).read()

        # Converti la maschera RGB nel formato appropriato per CrossEntropyLoss
        label = self.convert_labels_to_tensor(mask)

        # Restituisci l'immagine, la maschera e i parametri di georeferenziazione
        return image, label, world_params


In [6]:
# Definisci il percorso della cartella contenente i dati .tif e .tfw
data_folder = "/content/drive/MyDrive/PotsdamDataset"

# Crea un'istanza del dataset
dataset = PotsdamDataset(data_folder)

size = len(dataset)
train_size = int(0.7 * size)
val_size = int(0.15 * size)
test_size = int(size - train_size - val_size)

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

batch_size = 4

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [7]:
##
num_channels = 4  # RGBIR has 4 channels
dtype = torch.float32  # Imposta il tipo di dato desiderato

# placeholders
psum = torch.zeros(num_channels, dtype=dtype)
psum_sq = torch.zeros(num_channels, dtype=dtype)
count = 0

# loop through images
for inputs, labels, georeference_info in tqdm(train_loader):
    # Converti il tensore in tipo di dato a precisione maggiore
    inputs = inputs.to(dtype)

    psum += inputs.sum(axis = [0, 2, 3])
    psum_sq += (inputs ** 2).sum(axis = [0, 2, 3])

    # Conteggio dei pixel
    count += inputs.size(0) * inputs.size(2) * inputs.size(3)

total_mean = psum / count

# Calcola la varianza per canale
total_var = (psum_sq / count) - (total_mean ** 2)

# Calcola la deviazione standard per canale
total_std = torch.sqrt(total_var)
# output
print('mean per channel: ' + str(total_mean))
print('std per channel: ' + str(total_std))


100%|██████████| 7/7 [07:42<00:00, 66.09s/it]

mean per channel: tensor([87.4118, 93.3483, 86.6678, 98.1915])
std per channel: tensor([35.9870, 35.5850, 36.9591, 36.5149])


In [8]:
normalize_transform = transforms.Normalize(mean=total_mean, std=total_std)

# Funzione per convertire e normalizzare un'immagine
def normalize_batch(batch):
    images, labels, georeference_info = batch
    images = normalize_transform(images)
    return images, labels, georeference_info

# Crea nuovi DataLoader con la normalizzazione in batch
normalized_train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=normalize_batch)
normalized_val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=normalize_batch)
normalized_test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=normalize_batch)

In [ ]:
for img, _, _ in normalized_train_loader:
    print("Shape:", img.shape)
    print("Data type:", img.dtype)
    print("Example values:")
    print(img[0, 0, :10, :10])  # Mostra i primi 10x10 valori del primo canale
    break